# MVP Pipeline de Dados – Recursos Humanos  
## Análise de Retenção e Eficiência Organizacional  

**Vanessa Araújo**  

PUC-RJ – MBA em Ciência de Dados e Analytics  
Disciplina de Engenharia de Dados  

---

##Script para criação da base de dados SILVER

###Imports
Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType, StructField,
    StringType, IntegerType, DoubleType, DateType
)
from pyspark.sql.functions import col, to_date

###Criação da base de dados SILVER

Na camada Silver, os dados são persistidos após terem sido tratados, padronizados e estruturados, tornando-se adequados para análises analíticas e geração de insights.

Neste caso, a base original utilizada é o RHG – Human Resources Dataset, obtido no Kaggle.
Os dados foram inicialmente armazenados na camada Bronze, mantendo suas características originais.
Na camada Silver, são realizados tratamentos como:

Padronização de nomes de colunas

Conversão de tipos de dados

Criação de um modelo de dados analítico no formato estrela, apropriado para análises de Recursos Humanos

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS silver;

In [0]:
print("Iniciando criação da camada silver...")

# Iniciar uma sessão Spark
spark = SparkSession.builder.appName("CreateRHGTables").getOrCreate()

# Criar a tabela de dimensão "funcionário"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.dim_employee (
    employee_id STRING COMMENT 'Chave primária do funcionário',
    employee_name STRING COMMENT 'Nome do funcionário',
    gender STRING COMMENT 'Sexo do funcionário',
    race STRING COMMENT 'Raça declarada',
    marital_status STRING COMMENT 'Estado civil',
    citizenship STRING COMMENT 'Cidadania',
    hispanic_latino STRING COMMENT 'Identificação como Latino/Hispânico'
)
USING DELTA
""")

print("Tabela silver.dim_employee criada com sucesso.")

# Criar a tabela de dimensão "Departamento"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.dim_department (
    department_id STRING COMMENT 'Chave primária do departamento',
    department_name STRING COMMENT 'Nome do departamento'
)
USING DELTA
""")

print("Tabela silver.dim_department criada com sucesso.")

# Criar a tabela de dimensão "price"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.dim_position (
    position_id STRING COMMENT 'Chave primária do cargo',
    position_name STRING COMMENT 'Nome do cargo'
)
USING DELTA
""")

print("Tabela silver.dim_position criada com sucesso.")

# Criar a tabela de dimensão "Gestor"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.dim_manager (
    manager_id STRING COMMENT 'Chave primária do gestor',
    manager_name STRING COMMENT 'Nome do gestor'
)
USING DELTA
""")

print("Tabela silver.dim_manager criada com sucesso.")

# Criar a tabela de fatos "Funcionários"
spark.sql("""
CREATE TABLE IF NOT EXISTS silver.fact_employee (
    employee_id STRING COMMENT 'Chave estrangeira do funcionário',
    department_id STRING COMMENT 'Chave estrangeira do departamento',
    position_id STRING COMMENT 'Chave estrangeira do cargo',
    manager_id STRING COMMENT 'Chave estrangeira do gestor',
    hire_date DATE COMMENT 'Data de contratação',
    termination_date DATE COMMENT 'Data de desligamento',
    salary DOUBLE COMMENT 'Salário do funcionário',
    absences INTEGER COMMENT 'Quantidade de ausências',
    days_late_last_30 INTEGER COMMENT 'Atrasos nos últimos 30 dias',
    performance_score STRING COMMENT 'Avaliação de desempenho',
    employee_satisfaction INTEGER COMMENT 'Satisfação do funcionário',
    special_projects_count INTEGER COMMENT 'Quantidade de projetos especiais'
)
USING DELTA
""")

print("Tabela silver.fact_employee criada com sucesso.")
print("Tabelas criadas com sucesso!")

Iniciando criação da camada silver...
Tabela silver.dim_employee criada com sucesso.
Tabela silver.dim_department criada com sucesso.
Tabela silver.dim_position criada com sucesso.
Tabela silver.dim_manager criada com sucesso.
Tabela silver.fact_employee criada com sucesso.
Tabelas criadas com sucesso!


###Teste de Criação das Tabelas

Teste de acesso às tabelas recém-criadas para validação da estrutura.

In [0]:
spark.sql("DESCRIBE TABLE silver.dim_employee").show()
spark.sql("DESCRIBE TABLE silver.dim_department").show()
spark.sql("DESCRIBE TABLE silver.dim_position").show()
spark.sql("DESCRIBE TABLE silver.dim_manager").show()
spark.sql("DESCRIBE TABLE silver.fact_employee").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|    employee_id|   string|   NULL|
|  employee_name|   string|   NULL|
|         gender|   string|   NULL|
|           race|   string|   NULL|
| marital_status|   string|   NULL|
|    citizenship|   string|   NULL|
|hispanic_latino|   string|   NULL|
+---------------+---------+-------+

+---------------+---------+--------------------+
|       col_name|data_type|             comment|
+---------------+---------+--------------------+
|  department_id|   string|Chave primária do...|
|department_name|   string|Nome do departamento|
+---------------+---------+--------------------+

+-------------+---------+--------------------+
|     col_name|data_type|             comment|
+-------------+---------+--------------------+
|  position_id|   string|Chave primária do...|
|position_name|   string|       Nome do cargo|
+-------------+---------+--------------------+

+------------+---------+-